方策の微小な増分更新をおこなうアルゴリズムとしてCPIなどがありますが、ここでは、$\mu$から始まって、方策に従った場合の**軌道分布のKLダイバージェンス**を一定以下にする制約をつける制約付き最適化問題を考えます。この制約を**信頼領域制約**といいます。これによって、局所的な方策探索、つまり方策の微小な増分更新を促すことが可能になるようです。詳しく見ていきましょう。

#### 信頼領域制約付き最適化問題
$$
\begin{aligned}
& \max _\theta \mathbb{E}_{s \sim d_\mu^{\pi^{\theta_t}}} \mathbb{E}_{a \sim \pi_\theta(\cdot \mid s)} A^{\pi^{\theta_t}}(s, a) \\
& \text { s.t., } K L\left(\operatorname{Pr}_\mu^{\pi^{\theta_t}} \| \operatorname{Pr}_\mu^{\pi_\theta}\right) \leq \delta,
\end{aligned}
$$
そもそも、この最適化問題は何を目的としているのでしょうか？価値関数の最大化ではなく、アドバンテージ関数の最大化を考えています。これを考えるために、パフォーマンス補題を思い出します。


パフォーマンス補題は、更新後と更新前の方策に従う価値関数の差分を表しており、アドバンテージ関数を用いて表現できます。つまりアドバンテージ関数が最大化されればパフォーマンスの差分が最大化されます。この最大化は最も価値のある改善方向を選ぶという意味で最速で良くなることを示しています。

#### 最適化問題を二次計画法で表す
CPIは、方策の更新に対して保守的、つまり制約のようなものを与えてアドバンテージ関数を最大化させるのが目的でした。TRPOは、信頼領域を用いた制約を活用します。制約付き最適化問題に落としこむ場合、その界隈で扱われている手法を活用することを考えます。保守的な最適化を行うため、パラメータの更新量は微小です。そこで、目的関数と制約をパラメータの近傍?で近似テイラー展開することが可能であり、展開した関数と制約を使って局所的な二次計画法を作ることができます。
$$
\begin{aligned}
& \max _\theta \mathbb{E}_{s \sim d_\mu^{\pi^{\theta_t}}} \mathbb{E}_{a \sim \pi_\theta(\cdot \mid s)} A^{\pi^{\theta_t}}(s, a) \\
& \text { s.t., } K L\left(\operatorname{Pr}_\mu^{\pi^{\theta_t}} \| \operatorname{Pr}_\mu^{\pi_\theta}\right) \leq \delta,
\end{aligned}
$$
$$
\Downarrow
$$
$$
\begin{aligned}
& \max _\theta\left\langle\mathbb{E}_{s \sim d_\mu^{\pi^{\theta_t}}} \mathbb{E}_{a \sim \pi_{\theta_t}(\cdot \mid s)} \nabla_\theta \ln \pi_{\theta_t}(a \mid s) A^{\pi^{\theta_t}}(s, a), \theta\right\rangle \\
& \text { s.t., }\left\langle\left.\nabla_\theta K L\left(\operatorname{Pr}_\mu^{\pi^{\theta_t}} \| \operatorname{Pr}_\mu^{\pi_\theta}\right)\right|_{\theta=\theta_t}, \theta-\theta_t\right\rangle+\frac{1}{2}\left(\theta-\theta_t\right)^{\top}\left(\left.\nabla_\theta^2 K L\left(\operatorname{Pr}_\mu^{\pi^{\theta_t}} \| \operatorname{Pr}_\mu^{\pi_\theta}\right)\right|_{\theta=\theta_t}\right)\left(\theta-\theta_t\right) \leq \delta,
\end{aligned}
$$

$\left.\nabla^2 K L\right|_{\theta=\theta_t}$ は $\theta_t$ で測定された KLダイバージェンスのヘッセ行列です。

実は、制約の近似は結構面白い性質を持っています。Claim14.5を見てみましょう。

### Claim14.5

ホライゾン$H$ の有限ホライゾンMDP を考えます。任意の固定された $\theta_t$ について、以下が成りちます。
$$
\begin{aligned}
& \left.\nabla_\theta K L\left(\operatorname{Pr}_\mu^{\pi^{\theta_t}} \| \operatorname{Pr}_\mu^{\pi_\theta}\right)\right|_{\theta=\theta_t}=0, \\
& \left.\nabla_\theta^2 K L\left(\operatorname{Pr}_\mu^{\pi^{\theta_t}} \| \operatorname{Pr}_\mu^{\pi_\theta}\right)\right|_{\theta=\theta_t}=H \mathbb{E}_{s, a \sim d^{\pi_{\theta_t}}} \nabla \ln \pi_{\theta_t}(a \mid s)\left(\nabla \ln \pi_{\theta_t}(a \mid s)\right)^{\top} .
\end{aligned}
$$
①KLダイバージェンスの$\theta$に関する一次勾配は0になります。
②KLダイバージェンスの$\theta$に関する二次勾配は**フィッシャー情報行列**と一致します。

#### 証明
①について、
$$
\begin{aligned}
K L\left(\operatorname{Pr}_\mu^{\pi^{\theta_t}} \| \operatorname{Pr}_\mu^{\pi_\theta}\right) & =\sum_\tau \operatorname{Pr}_\mu^{\pi^{\theta_t}}(\tau) \ln \frac{\operatorname{Pr}_\mu^{\pi^{\theta_t}}(\tau)}{\operatorname{Pr}_\mu^{\pi^\theta}(\tau)} \\
& =\sum_\tau \operatorname{Pr}_\mu^{\pi^{\theta_t}}(\tau)\left(\sum_{h=0}^{H-1} \ln \frac{\pi_{\theta_t}\left(a_h \mid s_h\right)}{\pi_\theta\left(a_h \mid s_h\right)}\right)\\
&=\sum_{h=0}^{H-1} \mathbb{E}_{s_h, a_h \sim \mathbb{P}_h^{\pi_{\theta_t}}} \ln \frac{\pi_{\theta_t}\left(a_h \mid s_h\right)}{\pi_\theta\left(a_h \mid s_h\right)} .
\end{aligned}
$$
ここで、$\operatorname{Pr}_\mu^\pi(\tau)=\mu\left(s_0\right) \prod_{h=0}^{H-1} \pi\left(a_h \mid s_h\right) P\left(s_{h+1} \mid s_h, a_h\right)$です。
* 二行目は$\operatorname{Pr}_\mu^\pi(\tau)$を代入しただけです。
* 三行目は、$\left(\sum_{h=0}^{H-1} \ln \frac{\pi_{\theta_t}\left(a_h \mid s_h\right)}{\pi_\theta\left(a_h \mid s_h\right)}\right)$を確率変数(厳密には$s_h,a_h$が確率変数)とみて、その期待値を示しています。期待値の性質より、$\sum$は外に移動しています。

次に、$\left.\nabla_\theta KL\left(\operatorname{Pr}_\mu^{\pi^{\theta_t}} \|\operatorname{Pr}_\mu^{\pi_\theta}\right)\right|_{\theta=\theta_t}$を求めます。
$$
\begin{aligned}
&\left. \nabla_{\theta} KL(\operatorname{Pr}_\mu^{\pi^{\theta_t}}|| \operatorname{Pr}_\mu^{\pi_\theta}) \right|_{\theta=\theta_t} \\
&= \left. \nabla_{\theta} \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \log \frac{\operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau)}{\operatorname{Pr}_\mu^{\pi_\theta}(\tau)} \right|_{\theta=\theta_t} \\
&\qquad \text{対数の性質 } \log(a/b) = \log a - \log b \text{ を利用} \\
&= \left. \nabla_{\theta} \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \left( \log \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) - \log \operatorname{Pr}_\mu^{\pi_\theta}(\tau) \right) \right|_{\theta=\theta_t} \\
&\qquad \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \text{ と } \log \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \text{ は } \theta \text{ に依存しないため、微分すると0になる} \\
&= \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \left. \nabla_{\theta} \left( - \log \operatorname{Pr}_\mu^{\pi_\theta}(\tau) \right) \right|_{\theta=\theta_t} \\
&= - \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \left. \nabla_{\theta} \log \operatorname{Pr}_\mu^{\pi_\theta}(\tau) \right|_{\theta=\theta_t} \\
&\qquad \text{軌跡確率 } \operatorname{Pr}_\mu^{\pi_\theta}(\tau) \text{ を代入} \\
&= - \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \left. \nabla_{\theta} \log \left( \mu(s_0) \prod_{h=0}^{H-1} \pi_\theta(a_h | s_h) P(s_{h+1} | s_h, a_h) \right) \right|_{\theta=\theta_t} \\
&\qquad \text{対数の性質 } \log(abc) = \log a + \log b + \log c \text{ を利用} \\
&= - \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \left. \nabla_{\theta} \left( \log \mu(s_0) + \sum_{h=0}^{H-1} \log \pi_\theta(a_h | s_h) + \sum_{h=0}^{H-1} \log P(s_{h+1} | s_h, a_h) \right) \right|_{\theta=\theta_t} \\
&\qquad \mu(s_0) \text{ と } P(s_{h+1} | s_h, a_h) \text{ は } \theta \text{ に依存しないため、微分すると0になる} \\
&= - \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \sum_{h=0}^{H-1} \left. \nabla_{\theta} \log \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t} \\
&\qquad \text{和の順序を入れ替え、期待値の形に書き換える} \\
&= - \sum_{h=0}^{H-1} \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) \left. \nabla_{\theta} \log \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t} \\
&\qquad \text{時刻 } h \text{ までの状態・行動対 } (s_h, a_h) \text{ の周辺分布 } \mathbb{P}_h^{\pi_{\theta_t}}(s_h, a_h) \text{ を使うと} \\
&\qquad \sum_{\tau} \operatorname{Pr}_\mu^{\pi_{\theta_t}}(\tau) f(s_h, a_h) = \mathbb{E}_{\tau \sim \operatorname{Pr}_\mu^{\pi_{\theta_t}}} [f(s_h, a_h)] = \mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}} [f(s_h, a_h)] \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left[ \left. \nabla_{\theta} \log \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t} \right] \\
&= \sum_{h=0}^{H-1} \mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left[ - \nabla_{\theta} \left. \ln \pi_\theta(a_h \mid s_h) \right|_{\theta=\theta_t} \right] \\
&\qquad \text{期待値を状態 } s_h \text{ と行動 } a_h \text{ で分解する: } \mathbb{P}_h^{\pi_{\theta_t}}(s_h, a_h) = \mathbb{P}_h^{\pi_{\theta_t}}(s_h) \pi_{\theta_t}(a_h|s_h) \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{s_h \sim \mathbb{P}_h^{\pi_{\theta_t}}(s_h)} \left[ \mathbb{E}_{a_h \sim \pi_{\theta_t}(\cdot | s_h)} \left[ \left. \nabla_{\theta} \log \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t} \right] \right] \\
&\qquad \text{Log-Derivative Trick} \\
&\qquad \mathbb{E}_{a_h \sim \pi_{\theta_t}(\cdot | s_h)} \left[ \left. \nabla_{\theta} \log \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t} \right] \\
&\qquad = \sum_{a_h} \pi_{\theta_t}(a_h | s_h) \left. \frac{\nabla_{\theta} \pi_\theta(a_h | s_h)}{\pi_\theta(a_h | s_h)} \right|_{\theta=\theta_t} \\
&\qquad = \sum_{a_h} \pi_{\theta_t}(a_h | s_h) \frac{\left. \nabla_{\theta} \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t}}{\pi_{\theta_t}(a_h | s_h)} \\
&\qquad = \sum_{a_h} \left. \nabla_{\theta} \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t} \\
&\qquad = \left. \nabla_{\theta} \sum_{a_h} \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t} \\
&\qquad = \left. \nabla_{\theta} 1 \right|_{\theta=\theta_t} \\
&\qquad = 0 \\
&\qquad \text{この結果を元の式に代入} \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{s_h \sim \mathbb{P}_h^{\pi_{\theta_t}}(s_h)} [0] \\
&= 0
\end{aligned}
$$
②について、
$$
\begin{aligned}
&\left. \nabla^2_{\theta} KL(\operatorname{Pr}_\mu^{\pi^{\theta_t}}|| \operatorname{Pr}_\mu^{\pi_\theta}) \right|_{\theta=\theta_t} \\
&= \left. \nabla_\theta \left( \nabla_{\theta} KL(\operatorname{Pr}_\mu^{\pi^{\theta_t}}|| \operatorname{Pr}_\mu^{\pi_\theta}) \right) \right|_{\theta=\theta_t} \\
&= \left. \nabla_\theta \left( - \sum_{h=0}^{H-1} \mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left[ \nabla_{\theta} \log \pi_\theta(a_h | s_h) \right] \right) \right|_{\theta=\theta_t} & (1) \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left[ \left. \nabla^2_{\theta} \log \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t} \right] & (2) \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left[ \left. \nabla_\theta \left( \frac{\nabla_\theta \pi_\theta(a_h|s_h)}{\pi_\theta(a_h|s_h)} \right) \right|_{\theta=\theta_t} \right] & (3) \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left[ \left. \left( \frac{\nabla^2_\theta \pi_\theta(a_h | s_h)}{\pi_\theta(a_h | s_h)} - \frac{(\nabla_\theta \pi_\theta(a_h | s_h)) (\nabla_\theta \pi_\theta(a_h | s_h))^\top}{\pi_\theta^2(a_h | s_h)} \right) \right|_{\theta=\theta_t} \right] & (4) \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{s_h \sim \mathbb{P}_h^{\pi_{\theta_t}}} \mathbb{E}_{a_h \sim \pi_{\theta_t}(\cdot|s_h)} \left[ \frac{\left. \nabla^2_\theta \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t}}{\pi_{\theta_t}(a_h | s_h)} - \left. \frac{(\nabla_\theta \pi_\theta(a_h | s_h)) (\nabla_\theta \pi_\theta(a_h | s_h))^\top}{\pi_{\theta_t}^2(a_h | s_h)} \right|_{\theta=\theta_t} \right] & (5) \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{s_h \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left( \mathbb{E}_{a_h \sim \pi_{\theta_t}(\cdot|s_h)} \left[ \frac{\left. \nabla^2_\theta \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t}}{\pi_{\theta_t}(a_h | s_h)} \right] - \mathbb{E}_{a_h \sim \pi_{\theta_t}(\cdot|s_h)} \left[ \left( \nabla_{\theta_t} \log \pi_{\theta_t}(a_h|s_h) \right) \left( \nabla_{\theta_t} \log \pi_{\theta_t}(a_h|s_h) \right)^\top \right] \right) & (6) \\
&= - \sum_{h=0}^{H-1} \mathbb{E}_{s_h \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left( 0 - \mathbb{E}_{a_h \sim \pi_{\theta_t}(\cdot|s_h)} \left[ \left( \nabla_{\theta_t} \log \pi_{\theta_t}(a_h|s_h) \right) \left( \nabla_{\theta_t} \log \pi_{\theta_t}(a_h|s_h) \right)^\top \right] \right) & (7) \\
&= \sum_{h=0}^{H-1} \mathbb{E}_{s_h \sim \mathbb{P}_h^{\pi_{\theta_t}}} \mathbb{E}_{a_h \sim \pi_{\theta_t}(\cdot|s_h)} \left[ \left( \nabla_{\theta_t} \log \pi_{\theta_t}(a_h|s_h) \right) \left( \nabla_{\theta_t} \log \pi_{\theta_t}(a_h|s_h) \right)^\top \right] \\
&= \sum_{h=0}^{H-1} \mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}} \left[ \left( \nabla_{\theta_t} \log \pi_{\theta_t}(a_h|s_h) \right) \left( \nabla_{\theta_t} \log \pi_{\theta_t}(a_h|s_h) \right)^\top \right] & (8) \\
\end{aligned}
$$

*   式(3)から(4): $f(\theta) = \nabla_\theta \pi_\theta$ と $g(\theta) = \pi_\theta$ の商 $f/g$ の微分法則（正確には $\log g$ の2階微分に関する関係式 $\nabla^2 (\log g) = \nabla (\nabla g / g) = \frac{\nabla^2 g}{g} - \frac{(\nabla g)(\nabla g)^\top}{g^2}$ ）を適用してます。
*   式(4)から(5): 期待値 $\mathbb{E}_{(s_h, a_h) \sim \mathbb{P}_h^{\pi_{\theta_t}}}[\cdot]$ を分解しただけです。
*   式(6)から(7):   
    $$
    \mathbb{E}_{a_h \sim \pi_{\theta_t}(\cdot|s_h)} \left[ \frac{\left. \nabla^2_\theta \pi_\theta(a_h | s_h) \right|_{\theta=\theta_t}}{\pi_{\theta_t}(a_h | s_h)} \right] = \nabla^2_{\theta_t} \sum_{a_h} \pi_{\theta_t}(a_h | s_h) = \nabla^2_{\theta_t} (1) = 0
    $$
    から$0$になります。

#### フィッシャー情報行列と幾何学


#### 